It's good to be back!

We're looking at a dataset on EVERY nursing home in the country that received compensation from the CARES act. For ow, we're going to be looking at New York state.

In [2]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt

In [3]:
nh_df = pd.read_csv('/Users/karthikanamboothiri/Desktop/Columbia_academics/CODE/Stories_data/Nursing_homes/data/Provider_Relief_Fund_COVID-19_Nursing_Home_Quality_Incentive_Program.csv')
nh_df.columns

Index(['CCN', 'Facility Name', 'Address', 'City', 'State', 'Zip Code',
       'Total Resident Weeks', 'Total Covid Infections',
       'Facility Infection Rate  Per 1000 Resident Weeks',
       'County Infection Rate  Per 1000 Resident Weeks',
       'Infection Performance Score', 'Mortality Adjustment',
       'Performance Month', 'Final Payment'],
      dtype='object')

The infection measure calculations are here- [https://www.hhs.gov/coronavirus/cares-act-provider-relief-fund/general-information/index.html].

In summary, **Infection Performance Score** is measured as *the amount by which their own infection rate is lower than their county's infection rate* and *total patient volume, as measured by resident-weeks*. In a given performance period, a facility's infection rate will be measured as their total number of COVID infections (not including COVID admissions) divided by their total count of resident-weeks reported.

**Mortality Measure** is calculated as the total number of COVID deaths resulting from in-facility infections and the total number of non-admission infections.

In [4]:
ny_nh_df = nh_df[nh_df['State']=='NY']
ny_nh_df.City.value_counts()

BRONX            61
BROOKLYN         59
ROCHESTER        38
NEW YORK         20
FAR ROCKAWAY     20
                 ..
MARGARETVILLE     1
NEWFANE           1
MONTOUR FALLS     1
LOWVILLE          1
WEST ISLIP        1
Name: City, Length: 268, dtype: int64

In [5]:
nyc = ['BRONX', 'BROOKLYN', 'QUEENS', 'MANHATTAN', 'STATEN ISLAND', 'HARLEM', 'NEW YORK', 'QUEENS VILLAGE, ']
queens = [11361, 11362, 11363, 11364, 11354, 11355, 11356, 11357, 11358, 11359, 11360, 11365, 11366, 11367, 11412, 11423, 11432, 11433, 11434, 11435, 11436, 11101, 11102, 11103, 11104, 11105, 11106, 11374, 11375, 11379, 11385, 11691, 11692, 11693, 11694, 11695, 11697, 11004, 11005, 11411, 11413, 11422, 11426, 11427, 11428, 11429, 11414, 11415, 11416, 11417, 11418, 11419, 11420, 11421, 11368, 11369, 11370, 11372, 11373, 11377, 11378]
nyc_df = ny_nh_df[ny_nh_df['City'].isin(nyc) | ny_nh_df['Zip Code'].isin(queens)]
nyc_df

,CCN,Facility Name,Address,City,State,Zip Code,Total Resident Weeks,Total Covid Infections,Facility Infection Rate Per 1000 Resident Weeks,County Infection Rate Per 1000 Resident Weeks,Infection Performance Score,Mortality Adjustment,Performance Month,Final Payment
5735,335015,SCHERVIER NURSING CARE CENTER,2975 INDEPENDENCE AVE,BRONX,NY,10463,1196,0,0.0,0.24,0.29,0.00,September,"$ 37,539.89"
5737,335019,REGEIS CARE CENTER,3200 BAYCHESTER AVENUE,BRONX,NY,10475,666,0,0.0,0.24,0.16,0.00,September,"$ 20,904.32"
5741,335027,VILLAGE CARE REHABILITATION AND NURSING CENTER,214 W HOUSTON STREET,NEW YORK,NY,10014,318,0,0.0,0.20,0.06,0.00,September,"$ 8,121.09"
5742,335028,THE CITADEL REHAB AND NURSING CTR AT KINGSBRIDGE,3400 -26 CANNON PLACE,BRONX,NY,10463,1101,0,0.0,0.24,0.27,0.17,September,"$ 40,601.60"
5743,335030,MOSHOLU PARKWAY NURSING & REHABILITATION CENTER,3356 PERRY AVENUE,BRONX,NY,10467,370,0,0.0,0.24,0.09,0.20,September,"$ 13,936.21"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15760,335804,THE PAVILION AT QUEENS FOR REHABILITATION & NR...,36 17 PARSONS BOULEVARD,FLUSHING,NY,11354,1239,0,0.0,0.48,0.60,0.07,October,"$ 81,999.92"
15763,335811,LINDEN CENTER FOR NURSING AND REHABILITATION,2237 LINDEN BOULEVARD,BROOKLYN,NY,11207,1226,0,0.0,0.62,0.76,0.18,October,"$ 114,879.97"
15767,335822,INCARNATION CHILDRENS CENTER INC,142 AUDOBON AVENUE,NEW YORK,NY,10032,60,0,0.0,0.41,0.02,0.00,October,"$ 3,155.52"
15780,335849,JEANNE JUGAN RESIDENCE,2999 SCHURZ AVENUE,BRONX,NY,10465,128,0,0.0,0.42,0.05,0.00,October,"$ 6,923.37"


In [6]:
nyc_df[nyc_df['Total Covid Infections']==1]

,CCN,Facility Name,Address,City,State,Zip Code,Total Resident Weeks,Total Covid Infections,Facility Infection Rate Per 1000 Resident Weeks,County Infection Rate Per 1000 Resident Weeks,Infection Performance Score,Mortality Adjustment,Performance Month,Final Payment
15419,335100,ISABELLA GERIATRIC CENTER INC,515 AUDUBON AVENUE,NEW YORK,NY,10040,2461,1,0.41,0.41,0.01,0.20,October,"$ 1,486.07"
15590,335470,BORO PARK CENTER FOR REHABILITATION AND HEALTH...,4915 10TH AVE,BROOKLYN,NY,11219,2223,1,0.45,0.62,0.38,0.20,October,"$ 58,809.66"
15626,335537,"RUTLAND NURSING HOME, INC",585 SCHENECTADY AVE,BROOKLYN,NY,11203,1929,1,0.52,0.62,0.20,0.12,October,"$ 28,650.76"
15694,335665,TERENCE CARDINAL COOKE HEALTH CARE CENTER,1249 FIFTH AVENUE,NEW YORK,NY,10029,2705,1,0.37,0.41,0.11,0.20,October,"$ 16,884.99"


In [10]:
ny_nh_df.shape

(907, 14)